In [1]:
import sqlite3
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import os
print(os.getcwd())

/Users/chris/Desktop/MachineLearning2023/Lego


In [3]:
conn = sqlite3.connect('sql/lego.db')

In [4]:
cursor = conn.cursor()

## colors

In [58]:
dfColors = pd.read_csv('RebrickableData/colors.csv')

In [59]:
dfColors

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f
...,...,...,...,...
258,1132,HO Sand Green,A0BCAC,f
259,1133,HO Tan,E4CD9E,f
260,1134,HO Titanium,616161,f
261,1135,Metal,A5ADB4,f


In [60]:
dfColors.dtypes

id           int64
name        object
rgb         object
is_trans    object
dtype: object

In [61]:
dfColors.isna().sum()

id          0
name        0
rgb         0
is_trans    0
dtype: int64

In [62]:
boolMap = {'f':0, 't':1}

In [68]:
list(dfColors['is_trans'].unique()) ==['f','t']

True

In [70]:
for col in dfColors.columns:
    if list(dfColors[col].unique()) == ['f','t']:
        dfColors[col] = dfColors[col].map(boolMap)

In [71]:
dfColors

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,0
1,0,Black,05131D,0
2,1,Blue,0055BF,0
3,2,Green,237841,0
4,3,Dark Turquoise,008F9B,0
...,...,...,...,...
258,1132,HO Sand Green,A0BCAC,0
259,1133,HO Tan,E4CD9E,0
260,1134,HO Titanium,616161,0
261,1135,Metal,A5ADB4,0


In [32]:
dfColors.dtypes

id           int64
name        object
rgb         object
is_trans     int64
dtype: object

In [24]:
createColorTableQuery = '''
    CREATE TABLE IF NOT EXISTS colors (
      id INTEGER PRIMARY KEY,
      name TEXT,
      rgb TEXT,
      is_trans INTEGER
    )
'''
cursor.execute(createColorTableQuery)

In [33]:
dfColors.to_sql('colors', conn, if_exists='append', index=False)

263

## elements

In [34]:
dfElements = pd.read_csv('RebrickableData/elements.csv')

In [35]:
dfElements

,element_id,part_num,color_id,design_id
0,6443403,2277c01pr0009,1,2277.0
1,6300211,67906c01,14,67908.0
2,4566309,2564,0,2564.0
3,4275423,53657,1004,53657.0
4,6194308,92926,71,28967.0
...,...,...,...,...
84754,6462830,973c28h11pr0001,15,76382.0
84755,6461994,51066,71,51066.0
84756,6462014,21849pr0007,47,105302.0
84757,6450670,98100pr0013,25,104763.0


In [36]:
createElementTableQuery = '''
    CREATE TABLE IF NOT EXISTS elements (
      element_id INTEGER PRIMARY KEY,
      part_num TEXT,
      color_id INTEGER,
      design_id INTEGER
    )
'''
cursor.execute(createElementTableQuery)

In [37]:
dfElements.to_sql('elements', conn, if_exists='append', index=False)

84759

## inventories

In [50]:
tableName = 'inventories'

In [51]:
dfInventories = pd.read_csv(f'RebrickableData/{tableName}.csv')

In [52]:
dfInventories

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1
3,15,1,5158-1
4,16,1,903-1
...,...,...,...
37596,200328,1,854291-1
37597,200329,1,854292-1
37598,200330,1,854290-1
37599,200331,1,854294-1


In [53]:
dfInventories.dtypes

id          int64
version     int64
set_num    object
dtype: object

In [54]:
dfInventories['version'].value_counts()

version
1     35964
2      1011
3       199
4       150
5       110
6        58
7        37
8        27
9        20
10       16
11        3
12        2
13        1
14        1
15        1
16        1
Name: count, dtype: int64

In [55]:
qry = f'''
    CREATE TABLE IF NOT EXISTS {tableName} (
      id INTEGER PRIMARY KEY,
      version INTEGER,
      set_num TEXT
    )
'''
cursor.execute(qry)

In [56]:
dfInventories.to_sql(tableName, conn, if_exists='append', index=False)

37601

## generalize

In [12]:
import glob
from pathlib import Path

In [13]:
dataFiles = glob.glob('RebrickableData/*.csv')

In [14]:
dataFiles[0]

'RebrickableData/elements.csv'

In [15]:
tableNames = [Path(filename).stem for filename in dataFiles]
tableNames

['elements',
 'part_categories',
 'inventories',
 'parts',
 'part_relationships',
 'inventory_sets',
 'minifigs',
 'inventory_parts',
 'colors',
 'inventory_minifigs',
 'sets',
 'themes']

In [33]:
tableName = tableNames[7]
tableName

'inventory_parts'

In [34]:
df = pd.read_csv(f'RebrickableData/{tableName}.csv')

In [35]:
df

,inventory_id,part_num,color_id,quantity,is_spare,img_url
0,1,48379c04,72,1,f,https://cdn.rebrickable.com/media/parts/photos...
1,1,48395,7,1,f,https://cdn.rebrickable.com/media/parts/photos...
2,1,stickerupn0077,9999,1,f,NaN
3,1,upn0342,0,1,f,https://cdn.rebrickable.com/media/parts/photos...
4,1,upn0350,25,1,f,NaN
...,...,...,...,...,...,...
1187301,200014,25376,4,1,f,https://cdn.rebrickable.com/media/parts/elemen...
1187302,200014,3626cpr9649,14,1,f,https://cdn.rebrickable.com/media/parts/photos...
1187303,200014,87990,484,1,f,https://cdn.rebrickable.com/media/parts/elemen...
1187304,200014,970c46,3,1,f,https://cdn.rebrickable.com/media/parts/elemen...


In [36]:
df.dtypes

inventory_id     int64
part_num        object
color_id         int64
quantity         int64
is_spare        object
img_url         object
dtype: object

In [48]:
df['img_url'].dropna().unique()

array(['https://cdn.rebrickable.com/media/parts/photos/1/48379c01-1-839cbcec-62de-4733-ba23-20f35f4dd5d5.jpg',
       'https://cdn.rebrickable.com/media/parts/photos/7/48395-7-b9152acf-2fa5-4836-a04d-5b7fd39c2406.jpg',
       'https://cdn.rebrickable.com/media/parts/photos/135/upn0342-135-cde3e1b7-1f79-40bf-9b79-46fcf5dbae96.jpg',
       ...,
       'https://cdn.rebrickable.com/media/parts/elements/6463580.jpg',
       'https://cdn.rebrickable.com/media/parts/elements/6467618.jpg',
       'https://cdn.rebrickable.com/media/parts/elements/6468374.jpg'],
      dtype=object)

In [19]:
hasPrimaryKey = True

In [20]:
boolMap = {'f':0, 't':1}

In [24]:
np.sort(df[df.columns[0]].unique())

array([    9327,     9336,     9339, ..., 51815301, 60566614, 61532443])

In [49]:
for col in df.columns:
    print(list(np.sort(df[col].dropna().unique())) == ['f','t'])

False
False
False
False
True
False


In [52]:
for col in df.columns:
    if list(df[col].unique()) == ['f','t']:
        df[col] = df[col].map(boolMap)

In [53]:
typeDict = {np.dtype('int64'): 'INTEGER', np.dtype('object'): 'TEXT', np.dtype('float64'): 'REAL'}

In [54]:
def makeCreateTableQuery(df, tableName, hasPrimaryKey):
    qry = f'CREATE TABLE IF NOT EXISTS {tableName} (\n'
    for n, (col, type) in enumerate(df.dtypes.items()):
        if n==0 and hasPrimaryKey:
            qry += f'  {col} {typeDict[type]} PRIMARY KEY,\n'
        elif n==len(df.columns)-1:
            qry += f'  {col} {typeDict[type]}\n'
        else:
            qry += f'  {col} {typeDict[type]},\n'
    qry += ')'
    return qry

In [55]:
qry = makeCreateTableQuery(df, tableName,hasPrimaryKey)
print(qry)

CREATE TABLE IF NOT EXISTS inventory_parts (
  inventory_id INTEGER PRIMARY KEY,
  part_num TEXT,
  color_id INTEGER,
  quantity INTEGER,
  is_spare INTEGER,
  img_url TEXT
)


In [56]:

cursor.execute(qry)

In [57]:
df.to_sql(tableName, conn, if_exists='append', index=False)

84759

In [9]:
tableNames

['elements',
 'part_categories',
 'inventories',
 'parts',
 'part_relationships',
 'inventory_sets',
 'minifigs',
 'inventory_parts',
 'colors',
 'inventory_minifigs',
 'sets',
 'themes']

In [10]:
listHasPrimaryKey = ['elements',
 'part_categories',
 'inventories',
 'parts',
 'minifigs',
 'colors',
 'sets',
 'themes']

In [59]:
def MakeTableFromCSV(tableName):
    df = pd.read_csv(f'RebrickableData/{tableName}.csv')
    # map bool columns from 'f'/'t' to 0/1
    for col in df.columns:
        if list(np.sort(df[col].dropna().unique())) == ['f','t']:
            df[col] = df[col].map(boolMap)
    hasPrimaryKey = tableName in listHasPrimaryKey
    qry = makeCreateTableQuery(df, tableName, hasPrimaryKey)
    cursor.execute(qry)
    df.to_sql(tableName, conn, if_exists='append', index=False)

In [57]:
tableName

'inventory_parts'

In [30]:
MakeTableFromCSV(tableName)

In [60]:
MakeTableFromCSV('inventory_parts')

In [61]:
for tableName in tableNames:
    MakeTableFromCSV(tableName)

## define relationships

In [62]:
cursor.execute('PRAGMA foreign_keys = ON;')